# <B><u>Plotly/Matplotlib による Whale&Dolphin データセットのプレビュー</u></B>

# [1] はじめに

このノートブックでは、Kaggle Competition の "Happywhale - Whale and Dolphin Identification"のTraining(訓練)の方針やData augumentation(データ拡張)のやり方等々についての知見やヒントを得るため、<u>[Plotly](https://plotly.com/python/)</u>と<u>[Matplotlib](https://matplotlib.org/)</u>を用いて、データセットを可視化しています。参考として頂けますと幸いです。

このノートブックで紹介するデータ／図表の一覧を、以下に示します。

< Train images(訓練用の画像)に係るデータ／図表 >
- Train images(訓練用の画像)の数
- Species(種)の数
- Species(種)毎の、Train images(訓練用の画像)／Individuals(個体)の数をまとめた図表(Plotlyを使用)
- Individuals(個体)毎の、Train images(訓練用の画像)の数をまとめた図表(Plotlyを使用)
- Species(種)／Individuals(個体)毎の、Train images(訓練用の画像)のサンプル(Matplotlibを使用)

< Test images(検証用の画像)に係るデータ／図表 >
- Test images(検証用の画像)の数
- Test images(検証用の画像)のサンプル(Matplotlibを使用)

注意1：著者は、Kaggle／機械学習／Python の初心者です。したがって、内容は、初心者の方々に向けたものとなっています。また、内容に、バグや間違いが含まれている可能性があります。より良いノートブックとするため、忌憚のないご意見／コメントを頂けますと甚大です。

注意2：初心者の方々には、Kaggle Competition用のデータの読み込み、Pandasによるデータの処理、Numpy/Pillowによる画像データの取り扱い、Matplotlib/Plotlyによるデータの可視化 等々が参考になるかと思います。

注意3：このノートブックは、別途作成した<u>[英語のノートブック](https://www.kaggle.com/acchiko/preview-of-whale-dolphin-dataset-with-plotly-matpl?scriptVersionId=88631402)</u>を日本語に変更したものです。

# [2] データセットの準備

"Happywhale - Whale and Dolphin Identification" のデータセットは、ノートブックのサイドバーを開き、以下のボタンを順にクリックしていくと、(ノートブック内のコードから、)アクセスできるようになります。

###  "+ Add data" -> "Competition Data" -> "Add (Happywhale - Whale and Dolphin Identification)".

上記の手順がうまくいくと、以下のパスから、データにアクセスできるようになります。

In [ ]:
path_to_inputs = "/kaggle/input/happy-whale-and-dolphin"
!ls {path_to_inputs}

上記の、csvファイル(画像に関する情報をまとめたメタデータ)の内容を、以下に示します。

In [ ]:
!head {path_to_inputs}/sample_submission.csv

In [ ]:
!head {path_to_inputs}/train.csv

上記の、訓練用／検証用の画像のリストを、以下に示します。

In [ ]:
!ls {path_to_inputs}/train_images | head

In [ ]:
!ls {path_to_inputs}/test_images | head

上記の、訓練用／検証用の画像の数を、以下に示します。

In [ ]:
!echo "Number of train_images:"
!ls {path_to_inputs}/train_images | cat -n | tail -1 | cut -f1
!echo ""
!echo "Number of test_images:"
!ls {path_to_inputs}/test_images | cat -n | tail -1 | cut -f1

訓練用の画像については、csvファイル(画像に関する情報をまとめたメタデータ)が用意されていますが、検証の画像については、該当のファイルが用意されていないため、新たに作成します。

In [ ]:
# Generates metadata for test images. 
path_to_test_metadata = "/kaggle/working/test.csv"

!echo "image,species,individual_id" > {path_to_test_metadata}
!ls {path_to_inputs}/test_images | sed "s/.jpg/.jpg,unknown,unknown/g" >> {path_to_test_metadata}

# Shows contents of generated metadata.
!head {path_to_test_metadata}

# [3] Train images(訓練用の画像)の確認

Train images(訓練用の画像)と以下の統計データを示します。
- Species(種)の数
- Species(種)毎の、Train images(訓練用の画像)／Individuals(個体)の数をまとめた図表(Plotlyを使用)
- Individuals(個体)毎の、Train images(訓練用の画像)の数をまとめた図表(Plotlyを使用)

## [3-1] Train images(訓練用の画像)に係るデータセットの読み込み

In [ ]:
# Installs required libraries.
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install Pillow
!pip install plotly

In [ ]:
# Import required libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#import IPython

In [ ]:
# Defines the class to load metadata and images and to process those.
class WhaleAndDolphin():
    def __init__(self, path_to_metadata, path_to_dir_images):
        self._path_to_metadata = path_to_metadata
        self._path_to_dir_images = path_to_dir_images
        self._metadata = pd.read_csv(path_to_metadata)
        
    def getAllSpecies(self):
        return self._metadata["species"].unique()
    
    def sliceMetadata(self, query):
        return self._metadata.query(query).reset_index(drop=True)
    
    def getAllIndividualIDs(self, metadata):
        return metadata["individual_id"].unique()
    
    def showImagesTile(self, metadata, num_cols=4):
        num_rows = len(metadata) // num_cols + 1
        fig = plt.figure(figsize=(6.4 * num_cols, 4.8 * num_rows))
        
        for row in metadata.itertuples():
            ax = fig.add_subplot(num_rows, num_cols, row.Index + 1)
            
            title = self.getTitle(row)
            ax.set_title(title)
            
            image = self.getImage(row)
            plt.imshow(image)
            
        plt.show()
        plt.clf()
        plt.close()
        
    def getTitle(self, metadata_row):
        #return self._title(metadata_row.individual_id, metadata_row.species)
        return metadata_row.image
    
    def getImage(self, metadata_row):
        path_to_image = self._pathToImage(self._path_to_dir_images, \
                                          metadata_row.image)
        return Image.open(path_to_image)
    
    def _title(self, individual_id, species):
        return "%s (%s)" % (individual_id, species)
    
    def _pathToImage(self, path_to_dir_images, file_name):
        return "%s/%s" % (path_to_dir_images, file_name)
    
    def getImageArray(self, metadata_row):
        image_pil = self.getImage(metadata_row)
        return np.array(image_pil)
    
    def showIndividualImagesTile(self, metadata, num_cols=3, \
                                 max_num_individual_images=3, \
                                 max_num_individuals=10):
        individual_ids = self.getAllIndividualIDs(metadata)
        for individual_id in individual_ids[:max_num_individuals]:
            print()
            print("Individual ID : %s" % individual_id)
            metadata_individual = \
                metadata.query("individual_id == @individual_id").reset_index(drop=True)
            self.showImagesTile(
                metadata=metadata_individual[:max_num_individual_images], \
                num_cols=num_cols \
            )

訓練用の画像に係るデータセットを読み込みます。検証用の画像に係るデータセットは、後段で、同じクラスを使って読み込みます。

In [ ]:
# Loads metadata for train images.
path_to_metadata = "%s/train.csv" % path_to_inputs
path_to_dir_images = "%s/train_images" % path_to_inputs

whale_and_dolphin = WhaleAndDolphin(
    path_to_metadata=path_to_metadata,
    path_to_dir_images=path_to_dir_images
)

## [3-2] Train images(訓練用の画像)に係る統計データ

### [3-2-1] Species(種)の数

Species(種)の数を示します。

In [ ]:
# Shows number of species and its names.
all_species = whale_and_dolphin.getAllSpecies()

print("Number of species:")
print(len(all_species))
print()

print("Name of species:")
print(all_species)
print()

### [3-2-2] Species(種)毎の、Train images(訓練用の画像)／Individuals(個体)の数

Plotlyを用いて、Species(種)毎の、Train images(訓練用の画像)／Individuals(個体)の数をまとめた図表を示します。

In [ ]:
# Calculates numbers of images/individuals for each species are calculated.
metadata = {}
stats_species = pd.DataFrame(columns=["num_of_images", "num_of_individuals"], \
                             index=all_species)

for species in all_species:
    # Calculates number of images for each species.
    metadata[species] = whale_and_dolphin.sliceMetadata(query="species == @species")
    num_images = len(metadata[species])
    
    # Calculates number of individuals for each species. 
    individual_ids = whale_and_dolphin.getAllIndividualIDs(metadata[species])
    num_individuals = len(individual_ids)
    
    # Appends the result into summary table.
    stats_species.loc[species] = [num_images, num_individuals]

# Calculates total number of images/individuals and appends the result into summary table.
stats_species.loc["total"] = [stats_species["num_of_images"].sum(), stats_species["num_of_individuals"].sum()]

In [ ]:
# Shows the graph of numbers of images/individuals for each species with Plotly.
fig = go.Figure()
for column_name, items in stats_species[:len(stats_species)-1].iteritems():
    trace = go.Bar(x=items.index.tolist(), y=items.tolist(), name=column_name)
    fig.add_trace(trace)
fig.update_layout(yaxis_title="Number of images/individuals for each species")
fig.show()

In [ ]:
# Shows the table of numbers of images/individuals for each species.
print("Number of images/individuals for each species:")
stats_species

### [3-2-3] Individuals(個体)毎の、Train images(訓練用の画像)の数をまとめた図表

Plotlyを用いて、Individuals(個体)毎の、Train images(訓練用の画像)の数をまとめた図表を示します。画像の数が多く、全てを表示するのは難しいため、ここでは、例として、"false_killer_whale"のデータを示しています。必要に応じ、変数"species"の値を変えると、他の種のデータも表示することができます。

In [ ]:
species = "false_killer_whale" # It can be changed to "melon_headed_whale", "humpback_whale", etc.
individual_ids = whale_and_dolphin.getAllIndividualIDs(metadata[species])

stats_individuals = pd.DataFrame(columns=["num_of_images"], index=individual_ids)

for individual_id in individual_ids:
    # Calculates number of images for each individual and appends the result into summary table.
    metadata_individual = metadata[species].query("individual_id == @individual_id").reset_index(drop=True)
    num_images = len(metadata_individual)
    
    stats_individuals.loc[individual_id] = [num_images]

# Calculates total number of images/individuals and appends the result into summary table.
stats_individuals.loc["total"] = [stats_individuals["num_of_images"].sum()]

In [ ]:
# Shows the graph of numbers of images for each individuals with Plotly.
fig = go.Figure()
for column_name, items in stats_individuals[:len(stats_individuals)-1].iteritems():
    trace = go.Bar(x=items.index.tolist(), y=items.tolist(), name=column_name)
    fig.add_trace(trace)
fig.update_layout(title_text="Species : %s" % species, yaxis_title="Number of images for each individuals")
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
# Unsets limitation of display of rows.
pd.set_option("display.max_rows", None)

# Shows the table of number of images for each individuals.
print("Number of images for each individual of %s:" % species)
print("(Total number of individuals for %s: %d)" % (species, len(stats_individuals)-1))
stats_individuals

## [3-3] Train images(訓練用の画像)のサンプル

Plotlyを用いて、全てのSpecies(種)についての、最初の10の個体の、Train images(訓練用の画像)を示します。同じ個体の画像は、同じ行にまとめるようにしています。表示する画像の数は、num_cols等の変数の値を変えることで、調整することができます。

In [ ]:
# Shows train images for the first 10 individuals for each species.
num_cols = 4
max_num_individual_images = 4
max_num_individuals = 10

for species in all_species:
    print()
    print("--------------------------------------------------")
    print()
    print("   Images for %s" % species)
    print()
    print("--------------------------------------------------")
    whale_and_dolphin.showIndividualImagesTile( \
        metadata=metadata[species], \
        num_cols=num_cols, \
        max_num_individual_images=max_num_individual_images, \
        max_num_individuals=max_num_individuals \
    )
    print()
    print()

# [4] Test images(検証用の画像)の確認

Test images(検証用の画像)と以下の統計データを示します。
- Test images(検証用の画像)の数

## [4-1] Test images(検証用の画像)に係るデータセットの読み込み

 Test images(検証用の画像)は、Train images(訓練用の画像)の読み込みに用いたクラスで読み込むことができます。

In [ ]:
# Loads metadata for test images.
path_to_metadata = "%s" % path_to_test_metadata
path_to_dir_images = "%s/test_images" % path_to_inputs

whale_and_dolphin = WhaleAndDolphin(
    path_to_metadata=path_to_metadata,
    path_to_dir_images=path_to_dir_images
)

## [4-2] Test images(検証用の画像)に係る統計データ

Species(種)の数を示します。

In [ ]:
# Shows number of images for each species.
all_species = whale_and_dolphin.getAllSpecies()

print("Number of species:")
print(len(all_species))
print()

print("All species:")
print(all_species)
print()

Test images(検証用の画像)の数を示します。

In [ ]:
print("Number of images:")
print()
print("species, num_of_images")

metadata = {}
for species in all_species:
    metadata[species] = whale_and_dolphin.sliceMetadata(query="species == @species")
    num_images = len(metadata[species])
    
    print("%s, %d" % (species, num_images))

## [4-3] Test images(検証用の画像)のサンプル

Matplotlibを用いて、最初の100の、Test images(検証用の画像)を示します。表示する画像の数は、num_images等の変数の値を変えることで、調整することができます。

In [ ]:
# Shows first 100 test images.
num_images = 100
i_first = 0
i_end = i_first + num_images
num_cols = 4

for species in all_species:
    print()
    print("--------------------------------------------------")
    print()
    print("   Images for %s" % species)
    print()
    print("--------------------------------------------------")
    print()
    whale_and_dolphin.showImagesTile( \
        metadata=metadata[species][i_first:i_end], \
        num_cols=num_cols \
    )
    print()
    print()